In [3]:
# === ノートブック用：CSVのエンコード変換して上書き ===
from __future__ import annotations
import os, pathlib, tempfile, shutil
from typing import Optional, Iterable, List, Tuple

def _detect_encoding(data: bytes) -> Optional[str]:
    """
    chardet / charset-normalizer があれば推定に使う（任意）。
    未導入でもエラーにはしないで None を返す。
    """
    try:
        import chardet  # pip install chardet
        res = chardet.detect(data)
        enc = (res.get("encoding") or "").strip().lower()
        return enc or None
    except Exception:
        pass
    try:
        from charset_normalizer import from_bytes  # pip install charset-normalizer
        best = from_bytes(data).best()
        if best and best.encoding:
            return best.encoding.strip().lower()
    except Exception:
        pass
    return None

def transcode_csv_inplace(
    path: str | os.PathLike,
    src_encoding: Optional[str] = None,
    dst_encoding: str = "utf-8",
    add_bom: bool = False,
    newline: str = "\n",
    make_backup: bool = False,
) -> str:
    """
    CSVファイルのエンコードを変換して上書き（原子的置換）。
    Returns: 使われた実際のソースエンコード名（例: 'cp932', 'utf-8' など）

    Args:
      path: 変換対象のCSVパス
      src_encoding: 入力のエンコーディング。Noneなら推定→代表的候補で総当たり
      dst_encoding: 出力のエンコーディング（例: 'utf-8', 'cp932', 'utf-8-sig' など）
      add_bom: TrueならUTF-8でBOM付与（dst_encodingがutf-8-sigと等価）
      newline: 出力改行。'\n' or '\r\n'
      make_backup: Trueで <元ファイル>.bak を作成
    """
    p = pathlib.Path(path)
    if not p.is_file():
        raise FileNotFoundError(f"File not found: {p}")

    raw = p.read_bytes()

    # ソースエンコーディング決定
    used_src = (src_encoding or "").strip().lower() or _detect_encoding(raw)
    tried: List[str] = []
    text = None

    def try_decode(enc: str) -> Optional[str]:
        nonlocal text
        try:
            text = raw.decode(enc)
            return enc
        except Exception:
            tried.append(enc)
            return None

    if used_src:
        used_src = try_decode(used_src)

    if not used_src:
        # 日本語系・汎用系を順に試す
        for enc in ("cp932", "shift_jis", "euc_jp", "iso2022_jp", "utf-8", "utf-16", "latin-1"):
            if try_decode(enc):
                used_src = enc
                break

    if not used_src or text is None:
        raise UnicodeDecodeError("unknown", raw, 0, 1, f"decode failed; tried={tried}")

    # 出力側エンコーディング
    out_enc = dst_encoding.strip().lower()
    if add_bom and out_enc in ("utf-8", "utf8", "utf_8"):
        out_enc = "utf-8-sig"
    if out_enc in ("utf-8-sig", "utf_8_sig"):
        add_bom = True  # 表示用の一貫性

    # 改行正規化
    if newline not in ("\n", "\r\n"):
        raise ValueError("newline must be '\\n' or '\\r\\n'")
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    if newline == "\r\n":
        text = text.replace("\n", "\r\n")

    # 一時ファイルへ書き出し→原子的置換
    tmp_fd, tmp_path = tempfile.mkstemp(prefix=p.name + ".", dir=str(p.parent))
    try:
        with os.fdopen(tmp_fd, "w", encoding=out_enc, newline="") as f:
            f.write(text)

        if make_backup:
            shutil.copy2(p, p.with_suffix(p.suffix + ".bak"))

        os.replace(tmp_path, p)  # atomic
    except Exception:
        try:
            os.unlink(tmp_path)
        except Exception:
            pass
        raise

    return used_src

def batch_transcode_csv_inplace(
    paths: Iterable[str | os.PathLike],
    **kwargs,
) -> List[Tuple[str, str]]:
    """
    複数ファイルを一括変換。戻り値は (path, used_src_encoding) のリスト。
    kwargs は transcode_csv_inplace と同じ。
    """
    results: List[Tuple[str, str]] = []
    for path in paths:
        used = transcode_csv_inplace(path, **kwargs)
        results.append((str(path), used))
    return results


In [7]:
# 単一ファイル：自動推定 → UTF-8(BOMなし)で上書き
path = '/Users/araya/Library/Mobile Documents/com~apple~CloudDocs/Downloads/Araya/persona_intelligence/materials/datasets/dataverse_files/JHPSDGs20192020_06012022/householdsurvey06012022withJapaneselabel.csv'

# 文字化けするCSVをUTF-8(BOM付き) + CRLFで上書き
used = transcode_csv_inplace(
    path,
    src_encoding=None,     # 不明なら自動推定。分かっていれば "cp932" などを指定
    add_bom=True,          # これがポイント（BOM付与）
    newline="\r\n",        # Excel for Macで無難なCRLF
    make_backup=True       # 念のためバックアップ
)
print("decoded as:", used)


decoded as: utf-8
